In [1]:
import os
import pandas as pd

In [2]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [3]:
folder("pcaps")

In [4]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [5]:
files_add=find_the_way('./','.pcap')
files_add

['./pcaps\\HTTP_DDoS_00001.pcap',
 './pcaps\\HTTP_DoS_00001.pcap',
 './pcaps\\TCP_DDoS_00001.pcap',
 './pcaps\\TCP_DoS_00001.pcap',
 './pcaps\\UDP_DDoS_00001.pcap',
 './pcaps\\UDP_DoS_00001.pcap']

In [7]:
for i in files_add:
    temp=i[8:]
    temp=temp.replace(" ","_")
    temp=temp.replace("\\","-")
    temp=f"./pcaps/{temp}"
    #os.rename(i,temp)
    print(temp)

./pcaps/HTTP_DDoS_00001.pcap
./pcaps/HTTP_DoS_00001.pcap
./pcaps/TCP_DDoS_00001.pcap
./pcaps/TCP_DoS_00001.pcap
./pcaps/UDP_DDoS_00001.pcap
./pcaps/UDP_DoS_00001.pcap


In [8]:
files_add=find_the_way('./','.pcap')
uzun=len(files_add)
uzun

6

In [9]:
df = pd.read_csv( "dataset_description.csv")
df

,No,File Name,Category,Sub-category,Rule
0,2,HTTP_DDoS_00001.pcap,DDoS,HTTP FlooD,(ip.src==192.168.100.147 || ip.src==192.168.10...
1,3,HTTP_DoS_00001.pcap,DoS,HTTP FlooD,(ip.src==192.168.100.147 || ip.src==192.168.10...
2,4,TCP_DDoS_00001.pcap,DDoS,SYN FLOOD,(ip.src==192.168.100.147 || ip.src==192.168.10...
3,5,TCP_DoS_00001.pcap,DoS,SYN FLOOD,(ip.src==192.168.100.147 || ip.src==192.168.10...
4,6,UDP_DDoS_00001.pcap,DDoS,UDP FLOOD,(ip.src==192.168.100.147 || ip.src==192.168.10...
5,7,UDP_DoS_00001.pcap,DoS,UDP FLOOD,(ip.src==192.168.100.147 || ip.src==192.168.10...


In [10]:
df.columns

Index(['No', 'File Name', 'Category', 'Sub-category', 'Rule'], dtype='object')

In [11]:
name=df['File Name']
rule=df['Rule']
cat=df['Category']
subcat=df['Sub-category']

In [12]:
name

0    HTTP_DDoS_00001.pcap
1     HTTP_DoS_00001.pcap
2     TCP_DDoS_00001.pcap
3      TCP_DoS_00001.pcap
4     UDP_DDoS_00001.pcap
5      UDP_DoS_00001.pcap
Name: File Name, dtype: object

In [13]:
for i in range(len(name)):
    add=str(cat[i])+"@"+str(subcat[i])+"_"
    add=add.replace(" ","_")
    command='tshark -Y \"'+str(rule[i])+"\" -r ./pcaps/"+str(name[i])+" -T fields -e frame.number > ./pcaps/attack_"+str(name[i])[:-4]+"csv"
    os.system(command)
    command='tshark -Y \"!('+str(rule[i])+")\" -r ./pcaps/"+str(name[i])+" -T fields -e frame.number > ./pcaps/normal_"+str(name[i])[:-4]+"csv"
    os.system(command)
    files_add=find_the_way('./','.pcap')
    if uzun==len(files_add):    
        print(command,"\n")
    uzun=len(files_add) 
 
    

tshark -Y "!((ip.src==192.168.100.147 || ip.src==192.168.100.148  || ip.src==192.168.100.149  || ip.src==192.168.100.150))" -r ./pcaps/HTTP_DDoS_00001.pcap -T fields -e frame.number > ./pcaps/normal_HTTP_DDoS_00001.csv 

tshark -Y "!((ip.src==192.168.100.147 || ip.src==192.168.100.148  || ip.src==192.168.100.149  || ip.src==192.168.100.150))" -r ./pcaps/HTTP_DoS_00001.pcap -T fields -e frame.number > ./pcaps/normal_HTTP_DoS_00001.csv 

tshark -Y "!((ip.src==192.168.100.147 || ip.src==192.168.100.148  || ip.src==192.168.100.149  || ip.src==192.168.100.150) && tcp.flags.syn == 1)" -r ./pcaps/TCP_DDoS_00001.pcap -T fields -e frame.number > ./pcaps/normal_TCP_DDoS_00001.csv 

tshark -Y "!((ip.src==192.168.100.147 || ip.src==192.168.100.148  || ip.src==192.168.100.149  || ip.src==192.168.100.150) && tcp.flags.syn == 1)" -r ./pcaps/TCP_DoS_00001.pcap -T fields -e frame.number > ./pcaps/normal_TCP_DoS_00001.csv 

tshark -Y "!((ip.src==192.168.100.147 || ip.src==192.168.100.148  || ip.src==192

# MERGE CSVS

In [14]:
name_list=find_the_way('./pcaps','.csv')

for i in name_list:
    boyut=os.path.getsize(i)
    if boyut==0:
        os.remove(i)
        
    

In [15]:
label_files=[]
for i in name_list:
    temp=i[15:]
    if temp not in label_files:
        label_files.append(temp)
    

In [16]:
for i in label_files:

    name=find_the_way('./pcaps',i)
    print(name)

    dfA = pd.read_csv(name[0], header=None)
    dfA=dfA.rename(columns={0: "PacketNumber"})
    dfA["Label"]=[1] * len(dfA)
    
    try:
        dfN = pd.read_csv(name[1], header=None)
        dfN=dfN.rename(columns={0: "PacketNumber"})    
        dfN["Label"]=[0] * len(dfN) 
        df = pd.concat([dfA,dfN])
        df=df.sort_values('PacketNumber')
        df.to_csv(f"./pcaps/{i}",  index=False)
        print(i,df.groupby("Label").size(),"\n\n\n")
        os.remove(f"./pcaps\\normal_{i}")
        os.remove(f"./pcaps\\attack_{i}")
    except:
        dfA.to_csv(f"./pcaps/{i}",  index=False)
        print(i,dfA.groupby("Label").size(),"\n\n\n")
        

        


['./pcaps\\attack_HTTP_DDoS_00001.csv', './pcaps\\normal_HTTP_DDoS_00001.csv']
HTTP_DDoS_00001.csv Label
0    298085
1     59876
dtype: int64 



['./pcaps\\attack_HTTP_DoS_00001.csv', './pcaps\\normal_HTTP_DoS_00001.csv']
HTTP_DoS_00001.csv Label
0    298598
1     66043
dtype: int64 



['./pcaps\\attack_TCP_DDoS_00001.csv', './pcaps\\normal_TCP_DDoS_00001.csv']
TCP_DDoS_00001.csv Label
0    441686
1    950141
dtype: int64 



['./pcaps\\attack_TCP_DoS_00001.csv', './pcaps\\normal_TCP_DoS_00001.csv']
TCP_DoS_00001.csv Label
0    338647
1    932345
dtype: int64 



['./pcaps\\attack_UDP_DDoS_00001.csv', './pcaps\\normal_UDP_DDoS_00001.csv']
UDP_DDoS_00001.csv Label
0      44054
1    2295140
dtype: int64 



['./pcaps\\attack_UDP_DoS_00001.csv', './pcaps\\normal_UDP_DoS_00001.csv']
UDP_DoS_00001.csv Label
0      52901
1    2209912
dtype: int64 





In [105]:
#!shutdown /s /t 36

In [106]:
df

,PacketNumber,Label
0,1,1
0,2,0
1,3,0
1,4,1
2,5,0
...,...,...
1657,4971,1
3313,4972,0
3314,4973,0
3315,4974,0
